In [1]:
# Multiple AGN Catalog (MAC) - SMBH Binary SDSS/Optical Sources
# Author: R. W. Pfeifle
# Original Date Created: 10 Sept. 2020
# Last Revision Date: 7 August

# New Form Creation Date: 31 January 2022
# Last Revision: 26 September 2023

# Purpose: Combine various catalogs of double-peaked optically selected dual AGN candidates
# 

In [2]:
# Load in packages for pandas, astropy, etc. 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
from astropy.io import ascii
from astropy.table import Column, MaskedColumn
from astropy.io.ascii import masked
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.cosmology import LambdaCDM 
from astroquery.simbad import Simbad
from astroquery.sdss import SDSS

cosmo = LambdaCDM(H0=70, Om0=0.3, Ode0=0.7) #Creating our choice of cosmology here...

pd.set_option('display.max_columns', 300) # Setting max number of rows per df to be the size of the df
pd.set_option('display.max_rows', None)

In [3]:
def name_to_coords(df,dfcol):
    if (len(dfcol[0])) == 14:
        df['Coordinates'] = dfcol.str.slice(start=1) # Stripping the J
        df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=6) # Stripping the DEC parts 
        df['Dec_test'] = df['Coordinates'].str.slice(start=6, stop=13) # Stripping the RA parts
        df['RA'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=6) # Putting together the RA coordinates separated by colons
        df['Dec'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=8) # Putting together the Dec coodinates separated by colons
        df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
        return
    #print(dfcol.apply(len))
    elif (len(dfcol[0])) == 18:
        df['Coordinates'] = dfcol#.str.slice(start=1) # Stripping the J
        df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=9) # Stripping the DEC parts 
        df['Dec_test'] = df['Coordinates'].str.slice(start=9, stop=19) # Stripping the RA parts
        df['RA'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
        df['Dec'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
        df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
        return
    elif (len(dfcol[0])) == 19:
        df['Coordinates'] = dfcol.str.slice(start=1) # Stripping the J
        df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=9) # Stripping the DEC parts 
        df['Dec_test'] = df['Coordinates'].str.slice(start=9, stop=19) # Stripping the RA parts
        df['RA'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
        df['Dec'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
        df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
        return
    #elif dfcol.apply(len) ==
    else:
        print('Error Encountered')
        

In [4]:
# Here I'm rewriting our matching algorithm using the search_around_sky() function
# It may not always be the best option, but at least for these double peaked catalogs, I think I'm going to run 
# with it

def match_tables_fib(t1,t2,match_tol):
    if 'level_0' in t1.columns:
        t1.drop(labels=['level_0'], axis=1, inplace=True)
    t1.reset_index(drop=False, inplace=True)
    if 'level_0' in t2.columns:
        t2.drop(labels=['level_0'], axis=1, inplace=True)
    t2.reset_index(inplace=True, drop=False)
    t1['Table_flag'] = 'Table1'
    t2['Table_flag'] = 'Table2'
    # First we begin by matching RA1 and Dec1 of t1 to RA1 and Dec1 of t2
    c1 = SkyCoord(ra=t1['RA1_deg']*u.degree, dec=t1['Dec1_deg']*u.degree) # Storing coordinates for table 1
    c2 = SkyCoord(ra=t2['RA1_deg']*u.degree, dec=t2['Dec1_deg']*u.degree) # storing coordinates for table 2
    # Adding a match tolerance here, with user input for the function
    max_sep = match_tol * u.arcsec # The max match tolerance will be 5''
    #idx2, d2d2, d3d2 = match_coordinates_sky(c1, c2) # Now matching table 1 to table 2
    idx1, idx2, _, _ = c2.search_around_sky(c1, max_sep) 
    # idx1 and idx2 are the indices in table 1 and table 2 which are the closest matching rows to each other
    # Note, we should not need to cross match RA1 vs. RA2, across table because the double peaked sources only have
    # a single set of coordinates at this point
    # We need to make tables for t1 and t2 that do not include the matched items
    t1unique = (t1[~t1['index'].isin(idx1)]).reset_index(drop=True)
    t2unique = (t2[~t2['index'].isin(idx2)]).reset_index(drop=True)
    # And then we need a table for the matches items where we ensure they are properly matching (SDSS names should \
    # be the same), and then remove the duplicates, store the relevant info from the second table, and concatenate \
    # this with the primary table
    tmatches = pd.concat([(t1.iloc[idx1]),(t2.iloc[idx2])]).sort_values(by='Name').reset_index(drop=True)
    tunique = pd.concat([t1unique, t2unique]).sort_values(by='Name').reset_index(drop=True)
    #
    #t1matches.loc[t1matches['index'].isin(c1_dups['idx1']), 'Paper(s)'] += " ; " + t2['Paper(s)'][0]
    #t1matches.loc[t1matches['index'].isin(c1_dups['idx1']), 'BibCode(s)'] += " ; " + t2['BibCode(s)'][0]
    #t1matches.loc[t1matches['index'].isin(c1_dups['idx1']), 'DOI(s)'] += " ; " + t2['DOI(s)'][0]
    return tunique, tmatches, idx1, idx2




In [5]:
# Tsalmantza+2011 report the detection of 9 SMBH binary candidates
# Methodology: They search for systems exhibiting two sets of emission lines in the optical (using SDSS):
#    one broad and one narrow set of emission lines which are kinematically offset. A small separation would be 
#    expected in the case of Keplerian roation due to a binary. Here the narrow lines trace the galaxy, the broad
#    lines trace the active BH.
# Probes redshift 0.1-1.5, and applied fitting technique to SDSS QSO spectra as well as objects classified as galaxies
# All objects selected based on interesting features are at z<0.8
# There seems to be some overlap with Shen+2010 ****

# Loading in objects examined by Tsalmantza
tsalmantza = pd.read_csv('Tables/Tsalmantza2011/table1.csv', sep=',') #((Table.read('Tsalmantza2011/table1.dat', format='ascii')).to_pandas())#.drop(columns=['---'])

# Here, we're keeping only objects flagged as candidate binaries
tsalmantza = tsalmantza[(tsalmantza['Class']=='B') | (tsalmantza['Class']=='B;D') | (tsalmantza['Class']=='E;B') | (tsalmantza['Class']=='A;B') | (tsalmantza['Class']=='D;B') | (tsalmantza['Class']=='O;B')].reset_index(drop=True)

tsalmantza['Name1'] = tsalmantza['Name']
tsalmantza['Name2'] = tsalmantza['Name']
tsalmantza['z1'] = tsalmantza['z_NL']
tsalmantza['z2'] = tsalmantza['z_NL']
tsalmantza['z1_type'] = "spec"
tsalmantza['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(tsalmantza,tsalmantza['Name'])

# Converting the coordinates
coordconvert = SkyCoord(ra = tsalmantza['RA'], dec = tsalmantza['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
tsalmantza['RA1_deg'] = coordconvert.ra.degree
tsalmantza['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
tsalmantza['RA2'] = tsalmantza['RA']
tsalmantza['Dec2'] = tsalmantza['Dec']

tsalmantza['RA2_deg'] = tsalmantza['RA1_deg']
tsalmantza['Dec2_deg'] = tsalmantza['Dec1_deg']

# Adding details about the coordinates
tsalmantza['Equinox'] = "J2000"
tsalmantza['Coordinate_waveband'] = "Optical"
tsalmantza['Coordinate_Source'] = "SDSS"

tsalmantza['System Type'] = 'Binary AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
tsalmantza['Brightness1'] = -100
tsalmantza['Brightness_band1'] = -100
tsalmantza['Brightness_type1'] = -100

tsalmantza['Brightness2'] = -100
tsalmantza['Brightness_band2'] = -100
tsalmantza['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
tsalmantza['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit



#tsalmantza['Sep(kpc)'] = tsalmantza['Sep']*((cosmo.arcsec_per_kpc_proper(tsalmantza['z']))**(-1))



# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#tsalmantza['delta_z'] = tsalmantza['z1']-tsalmantza['z2']
tsalmantza['dV'] = (2.99e+5)*((1+tsalmantza['z1'])**2 - (1+tsalmantza['z2'])**2)/((1+tsalmantza['z1'])**2+(1+tsalmantza['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
tsalmantza['Selection Method'] = "Velocity Shifts between Narrow and Broad Lines" #DPSELs
tsalmantza['Confirmation Method'] = "-99"
tsalmantza['Paper(s)'] = "Tsalmantza+2011"
tsalmantza['BibCode(s)'] = "2011ApJ...738...20T"
tsalmantza['DOI(s)'] = "https://doi:10.1088/0004-637X/738/1/20"
tsalmantza['Notes'] = " "

## And dropping any columns that we don't need....
#tsalmantza.drop(labels=['SDSS','f_SDSS','Vel','logL','Type','Q','zr','zh'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#tsalmantza = tsalmantza[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#tsalmantza.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']

#tsalmantza




# Eracleous+2012 report the detection of 
# Redshift range of z<0.7. 
# Trying to probe binaries with separations of subparsecs and periods of a few centuries
# They systematically search SDSS for quasars whose broad (single-peaked) B-beta lines are offset from the expected
# wavelength by 1000 km/s or more, considering these as candidate SMBH binaries or recoiling SMBHs
#     They throw out systems where delta v < 1000 km/s, and systems showing clear DPE signatures.
#     But they retain flat top line profiles as well as line profiles that are displayed but have an extended wing or shoulder
# ---> They stress this is not a complete sample, and is very subjective given that visual classification was used, 
#      and they added two objects from Strateva+03
# ----> There is overlap with Tsalmantza+ and others

# Bonning+200 attempted to find binaries or recoils ...



In [6]:
# # Decarli+ was a direct follow-up work to Tsalmantza+

# # Here we're loading in the Decarli+2013 and Lusso+ tables, which we'll need to match on the decarli2013+2011 table since \
# # the sample in Decarli+2013 are pulled directly from the binary candidates in decarli2013+2011.

# #Summary of the follow-up observations. (1) Quasar name. (2) and (3) Right ascension and declination (J2000). \
# # (4) Object classification, based on the shape of Balmer lines: B – BHB candidates; D – DPEs; A – sources \
# # with Asymmetric line profiles; O – Others. (5) Redshift of narrow lines. (6) Date of spectroscopic \
# # follow-up observations. (7) Time lag between SDSS observations and our follow-up spectroscopy, in the \
# # rest frame of zNL. (8) Telescope: C2.2m = Calar Alto 2.2 m; C3.5m = Calar Alto 3.5 m; Lick; Keck. (9) Weather \
# # conditions during the spectroscopic follow-up: P = photometric; NP = not photometric; C = cloudy.

# decarli2013 = pd.read_csv('Tables/Decarli2013/Decarli2013.csv', sep=',')

# decarli2013 = decarli2013[decarli2013[()]]
# #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# # Here, we're keeping only objects flagged as candidate binaries
# #decarli2013 = decarli2013[(decarli2013['Class']=='B') | (decarli2013['Class']=='B;D') | (decarli2013['Class']=='E;B') | (decarli2013['Class']=='A;B') | (decarli2013['Class']=='D;B') | (decarli2013['Class']=='O;B')].reset_index(drop=True)

# decarli2013['Name1'] = decarli2013['Objname']
# decarli2013['Name2'] = decarli2013['Objname']
# decarli2013['z1'] = decarli2013['zNL']
# decarli2013['z2'] = decarli2013['zNL']
# decarli2013['z1_type'] = "spec"
# decarli2013['z2_type'] = "spec"

# # Now converting the naming convention to RA and Dec and adding some informative columns
# #name_to_coords(decarli2013,decarli2013['Name'])

# # Converting the coordinates
# coordconvert = SkyCoord(ra = decarli2013['RA'], dec = decarli2013['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
# decarli2013['RA1_deg'] = coordconvert.ra.degree
# decarli2013['Dec1_deg'] = coordconvert.dec.degree

# # Adding in a second set of coordinates for the 'secondary'
# decarli2013['RA2'] = decarli2013['RA']
# decarli2013['Dec2'] = decarli2013['Dec']

# decarli2013['RA2_deg'] = decarli2013['RA1_deg']
# decarli2013['Dec2_deg'] = decarli2013['Dec1_deg']

# # Adding details about the coordinates
# decarli2013['Equinox'] = "J2000"
# decarli2013['Coordinate_waveband'] = "Optical"
# decarli2013['Coordinate_Source'] = "SDSS"

# decarli2013['System Type'] = 'Binary AGN Candidate'

# # Adding in some columns that we'll population via a Simbad or Ned search later
# decarli2013['Brightness1'] = -100
# decarli2013['Brightness_band1'] = -100
# decarli2013['Brightness_type1'] = -100

# decarli2013['Brightness2'] = -100
# decarli2013['Brightness_band2'] = -100
# decarli2013['Brightness_type2'] = -100

# # Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# # of order ~1 kpc or less, but is not currently determined.
# decarli2013['Sep'] = 3 # arcseconds
# # Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# # fiber as an upper limit



# #decarli2013['Sep(kpc)'] = decarli2013['Sep']*((cosmo.arcsec_per_kpc_proper(decarli2013['z']))**(-1))



# # For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
# #decarli2013['delta_z'] = decarli2013['z1']-decarli2013['z2']
# decarli2013['dV'] = (2.99e+5)*((1+decarli2013['z1'])**2 - (1+decarli2013['z2'])**2)/((1+decarli2013['z1'])**2+(1+decarli2013['z2'])**2)
# # dV will be zero until we include follow-up observations that show separate redshifts

# # Adding information about the paper and the selection method
# decarli2013['Selection Method'] = "Velocity Shifts between Narrow and Broad Lines" #DPSELs
# decarli2013['Confirmation Method'] = "-99"
# decarli2013['Paper(s)'] = "Decarli+2013"
# decarli2013['BibCode(s)'] = "2013MNRAS.433.1492D"
# decarli2013['DOI(s)'] = "https://doi.org/10.1093/mnras/stt831"
# decarli2013['Notes'] = ' '
# ## And dropping any columns that we don't need....
# #decarli2013.drop(labels=['SDSS','f_SDSS','Vel','logL','Type','Q','zr','zh'],\
# #              axis=1, inplace=True)

# ## Rearranging the columns and renaming columns now...
# #decarli2013 = decarli2013[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
# #                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
# #                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
# #                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
# #                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
# #decarli2013.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
# #                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
# #                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
# #                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
# #                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']


# decarli2013


In [7]:
# # Here we'll add Lusso+2014, which was a follow-up to Tsalmantza+2011 and Decarli+2013

# #for index, row in decarli2013.iterrows():
#     #if row['Name'] in hutsemekers2020:
# decarli2013['Paper(s)'] += ' ; Lusso+2014 '
# decarli2013['BibCode(s)'] += ' ; 2014MNRAS.441..316L' 
# decarli2013['DOI(s)'] += ' ;  https://doi.org/10.1093/mnras/stu572'
# decarli2013['Notes']+=' Lusso+ fit the SEDs to search for evidence of peculiar properties that could be consistent with the recoil hypothesis. In a scenario where teh recoilis displaced from the center of the galaxy and has left the torus behind the AGN should show a deficit of 12um emission. They find that all objects in the sample (with the possible exception of J1154) exhibit a significant amount of 12um emission in contrast to the recoil scenario.'

# decarli2013

In [8]:
# Here we'll add Decarli+2013 and Lusso+2014, which was a follow-up to Tsalmantza+2011

tsalmantza['Paper(s)'] += ' ; Decarli+2013 ; Lusso+2014'
tsalmantza['BibCode(s)'] += ' ; 2013MNRAS.433.1492D ; 2014MNRAS.441..316L' 
tsalmantza['DOI(s)'] += ' ; https://doi.org/10.1093/mnras/stt831 ; https://doi.org/10.1093/mnras/stu572'
tsalmantza['Notes']+=' Decarli+ find most objects do not undergo significant profile or flux variation. Lusso+ fit the SEDs to search for evidence of peculiar properties that could be consistent with the recoil hypothesis. In a scenario where the recoils displaced from the center of the galaxy and has left the torus behind the AGN should show a deficit of 12um emission. They find that all objects in the sample (with the possible exception of J1154) exhibit a significant amount of 12um emission in contrast to the recoil scenario.'

# we may want to consider combing back and adding in additional in-text details from Decarli+

#tsalmantza

In [9]:
# Here we're loading in the Shen+2013 catalog and we are ONLY including objects with their binary candidate flag \
# indicated by the (1) flag in  the 'Category' column
# I am not including ambiguous (3) or variability (2) cases
shen2013 = pd.read_csv('Tables/Shen2013/table2.csv', sep=',')
shen2013 = shen2013[shen2013['Category']==1.0]

shen2013['Name1'] = shen2013['Designation']
shen2013['Name2'] = shen2013['Designation']
shen2013['z1'] = shen2013['z_sys']
shen2013['z2'] = shen2013['z_sys']
shen2013['z1_type'] = "spec"
shen2013['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(shen2013,shen2013['Designation'])

shen2013['Coordinates'] = shen2013['Designation']#.str.slice(start=1) # Stripping the J
shen2013['RA_test'] = shen2013['Coordinates'].str.slice(start=0, stop=9) # Stripping the DEC parts 
shen2013['Dec_test'] = shen2013['Coordinates'].str.slice(start=9, stop=19) # Stripping the RA parts
shen2013['RA'] = shen2013['RA_test'].str.slice(start=0, stop=2)+":"+shen2013['RA_test'].str.slice(start=2, stop=4)+":"+shen2013['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
shen2013['Dec'] = shen2013['Dec_test'].str.slice(start=0, stop=3)+":"+shen2013['Dec_test'].str.slice(start=3, stop=5)+":"+shen2013['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
shen2013.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)

# Converting the coordinates
coordconvert = SkyCoord(ra = shen2013['RA'], dec = shen2013['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
shen2013['RA1_deg'] = coordconvert.ra.degree
shen2013['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
shen2013['RA2'] = shen2013['RA']
shen2013['Dec2'] = shen2013['Dec']

shen2013['RA2_deg'] = shen2013['RA1_deg']
shen2013['Dec2_deg'] = shen2013['Dec1_deg']

# Adding details about the coordinates
shen2013['Equinox'] = "J2000"
shen2013['Coordinate_waveband'] = "Optical"
shen2013['Coordinate_Source'] = "SDSS"

shen2013['System Type'] = 'Binary AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
shen2013['Brightness1'] = -100
shen2013['Brightness_band1'] = -100
shen2013['Brightness_type1'] = -100

shen2013['Brightness2'] = -100
shen2013['Brightness_band2'] = -100
shen2013['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
shen2013['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit



#shen2013['Sep(kpc)'] = shen2013['Sep']*((cosmo.arcsec_per_kpc_proper(shen2013['z']))**(-1))



# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
shen2013['delta_z'] = shen2013['z1']-shen2013['z2']
shen2013['dV'] = (2.99e+5)*((1+shen2013['z1'])**2 - (1+shen2013['z2'])**2)/((1+shen2013['z1'])**2+(1+shen2013['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
shen2013['Selection Method'] = "Extreme Velocity Offsets / Radial Velocity Shifts" #DPSELs
shen2013['Confirmation Method'] = "-99"
shen2013['Paper(s)'] = "Shen+2013"
shen2013['BibCode(s)'] = "2013ApJ...775...49S"
shen2013['DOI(s)'] = "https://doi.org/10.1088/0004-637X/775/1/49"
shen2013['Notes'] = " "


## And dropping any columns that we don't need....
#shen2013.drop(labels=['SDSS','f_SDSS','Vel','logL','Type','Q','zr','zh'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#shen2013 = shen2013[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#shen2013.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']
#

shen2013.reset_index(drop=True, inplace=True)

#shen2013

In [10]:
# Creating an overarching catalog here now...
tunique, tmatches, idx1, idx2 = match_tables_fib(tsalmantza,shen2013,20)

# verified that there are no matches to within 20''
#print(len(tmatches))

# concatenating tsalmantza and shen here to make an overarching catalog

the_whills = pd.concat([tsalmantza,shen2013]).reset_index(drop=True)

#the_whills

In [11]:
# Loading in Liu+2014 (Paper II of their series, where Shen+2013 was Paper I)

# NOTE: The SDSS designations change between Tables 1 and 2 in Liu+2014. 
# This very likely means that the designations in Gou+ will only match one of these tables (or neither of them)
# I am not going to go through and fix this now. 


# For now, I am ONLY loading in a table of the specific binary candidates. I am not loading in their full sample!
liu2014 = ((Table.read('Tables/XLiu2014/table1.dat', readme = 'Tables/XLiu2014/ReadMe', format='ascii.cds')).to_pandas())#.drop(columns=['---'])

liu2014['Name'] = liu2014['SDSS']
liu2014['Name2'] = liu2014['SDSS']
name_to_coords(liu2014,liu2014['SDSS'])

# Converting the coordinates
coordconvert = SkyCoord(ra = liu2014['RA'], dec = liu2014['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
liu2014['RA1_deg'] = coordconvert.ra.degree
liu2014['Dec1_deg'] = coordconvert.dec.degree


liu2014['z1'] = liu2014['z']
liu2014['z2'] = liu2014['z']
liu2014['z1_type'] = "spec"
liu2014['z2_type'] = "spec"

# Adding details about the coordinates
liu2014['Equinox'] = "J2000"
liu2014['Coordinate_waveband'] = "Optical"
liu2014['Coordinate_Source'] = "SDSS"

liu2014['System Type'] = 'Binary AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
liu2014['Brightness1'] = -100
liu2014['Brightness_band1'] = -100
liu2014['Brightness_type1'] = -100

liu2014['Brightness2'] = -100
liu2014['Brightness_band2'] = -100
liu2014['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
liu2014['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit

#liu2014['Sep(kpc)'] = liu2014['Sep']*((cosmo.arcsec_per_kpc_proper(liu2014['z']))**(-1))

# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#liu2014['delta_z'] = liu2014['z1']-liu2014['z2']
liu2014['dV'] = (2.99e+5)*((1+liu2014['z1'])**2 - (1+liu2014['z2'])**2)/((1+liu2014['z1'])**2+(1+liu2014['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts


# Adding information about the paper and the selection method
liu2014['Selection Method'] = "Extreme Velocity Offsets / Radial Velocity Shifts" #DPSELs
liu2014['Confirmation Method'] = "-99"
liu2014['Paper(s)'] = "Liu+2014"
liu2014['BibCode(s)'] = "2014ApJ...789..140L"
liu2014['DOI(s)'] = "https://doi.org/10.1088/0004-637X/789/2/140"
liu2014['Notes'] = " "



#liu2014

In [12]:
liu2014t2 = pd.read_csv('Tables/Xliu2014/table2.csv', sep=',')
# Limiting to only the systems they flagged as binary candidates now....
#liu2014t2 = liu2014t2t2[liu2014t2t2['Category']=='1']

liu2014t2['Name'] = liu2014t2['SDSS Designation']
liu2014t2['Name2'] = liu2014t2['SDSS Designation']
#liu2014t2['z'] = -99

#liu2014t2.loc[liu2014t2['SDSS Designation']=='082930.60+272822.7', 'z']=0.3211
#liu2014t2.loc[liu2014t2['SDSS Designation']=='084716.04+373218.1', 'z']=0.4534
#liu2014t2.loc[liu2014t2['SDSS Designation']=='085237.02+200411.0', 'z']=0.4615
#liu2014t2.loc[liu2014t2['SDSS Designation']=='092837.98+602521.0', 'z']=0.2959
#liu2014t2.loc[liu2014t2['SDSS Designation']=='103059.09+310255.8', 'z']=0.1781
#liu2014t2.loc[liu2014t2['SDSS Designation']=='110051.02+170934.3', 'z']=0.3476
#liu2014t2.loc[liu2014t2['SDSS Designation']=='111230.90+181311.4', 'z']=0.1952
#liu2014t2.loc[liu2014t2['SDSS Designation']=='130534.49+181932.9', 'z']=0.1178
#liu2014t2.loc[liu2014t2['SDSS Designation']=='134548.50+114443.5', 'z']=0.1264

#liu2014t2['z1'] = liu2014t2['z']
#liu2014t2['z2'] = liu2014t2['z']
#liu2014t2['z1_type'] = "spec"
#liu2014t2['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(liu2014t2,liu2014t2['Designation'])

liu2014t2['Coordinates'] = liu2014t2['SDSS Designation']#.str.slice(start=1) # Stripping the J
liu2014t2['RA_test'] = liu2014t2['Coordinates'].str.slice(start=0, stop=9) # Stripping the DEC parts 
liu2014t2['Dec_test'] = liu2014t2['Coordinates'].str.slice(start=9, stop=19) # Stripping the RA parts
liu2014t2['RA'] = liu2014t2['RA_test'].str.slice(start=0, stop=2)+":"+liu2014t2['RA_test'].str.slice(start=2, stop=4)+":"+liu2014t2['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
liu2014t2['Dec'] = liu2014t2['Dec_test'].str.slice(start=0, stop=3)+":"+liu2014t2['Dec_test'].str.slice(start=3, stop=5)+":"+liu2014t2['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
liu2014t2.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)

# Converting the coordinates
coordconvert = SkyCoord(ra = liu2014t2['RA'], dec = liu2014t2['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
liu2014t2['RA1_deg'] = coordconvert.ra.degree
liu2014t2['Dec1_deg'] = coordconvert.dec.degree


#liu2014t2

In [13]:
tunique, tmatches, idx1, idx2 = match_tables_fib(liu2014,liu2014t2,5)

print(len(tmatches))

for i, j in zip(idx1, idx2):
    #the_whills.at[i, 'Sep'] = fu2011d.at[j, 'Sep_as']
    #the_whills.at[i, 'dV'] = fu2011d.at[j, 'dV']
    liu2014.at[i, 'Notes'] += ' Liu+2014 examined follow-up second-epoch spectra to monitor the velocity offset broad lines.'

liu2014t22 = liu2014t2[(liu2014t2['Category']=='1') | (liu2014t2['Category']=='2') | (liu2014t2['Category']=='3') | (liu2014t2['Category']=='4')]

tunique, tmatches, idx1, idx2 = match_tables_fib(liu2014,liu2014t22,5)

print(len(tmatches))
for i, j in zip(idx1, idx2):
    #the_whills.at[i, 'Sep'] = fu2011d.at[j, 'Sep_as']
    #the_whills.at[i, 'dV'] = fu2011d.at[j, 'dV']
    liu2014.at[i, 'Notes'] += 'Liu+2014 find significant radial acceleration in the broad Halpha line in this object.'

liu2014t22 = liu2014t2[liu2014t2['Category']=='1']

tunique, tmatches, idx1, idx2 = match_tables_fib(liu2014,liu2014t22,5)

print(len(tmatches))
for i, j in zip(idx1, idx2):
    #the_whills.at[i, 'Sep'] = 
    #the_whills.at[i, 'dV'] = fu2011d.at[j, 'dV']
    liu2014.at[i, 'Notes'] += 'Liu+2014 consider this to be a candidate sub-pc binary due to consistent velocity shifts independently observed in a second broad emission line.'


# verified that this matching process works


100
48
18


/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_99630/1688091587.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t2['Table_flag'] = 'Table2'
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_99630/1688091587.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t2['Table_flag'] = 'Table2'


In [63]:
#liu2014

In [14]:
# Now matching Liu+2014 to the the_whills
tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,liu2014,20)

print(len(tmatches))

for i, j in zip(idx1, idx2):
    the_whills.at[i, 'Paper(s)'] += ' ; Liu+2014'
    the_whills.at[i, 'BibCode(s)'] += ' ; 2014ApJ...789..140L' 
    the_whills.at[i, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-637X/789/2/140'
    the_whills.at[i, 'Notes'] += liu2014.at[j, 'Notes']

for i in the_whills['Paper(s)']:
    if 'Liu+2014' in i:
        print('True')

liu_nodups = liu2014.drop(idx2)

the_whills = pd.concat([the_whills,liu_nodups])

#the_whills

# verified that this entire process is working smoothly now


8
True
True
True
True


In [15]:
#tmatches

In [16]:
# tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,smith2012,5)

# # Adding the DOI, author, and bibcode info to all of the Smith+2010 rows here in the matches table...
# for index, row in tmatches.iterrows():
#     if row['Table_flag']!='Table2':
#         tmatches.at[index, 'Paper(s)'] += ' ; Smith+2012'
#         tmatches.at[index, 'BibCode(s)'] += ' ; 2012ApJ...752...63S' 
#         tmatches.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-637X/752/1/63'

# # Now clipping out all Smith+2010 rows from the matches table
# tmatches = tmatches[tmatches['Table_flag']!='Table2'].reset_index(drop=True)

# # Concatenating everything together to generate a master table here
# the_whills = pd.concat([tmatches,tunique]).sort_values(by='Name').reset_index(drop=True)
# the_whills.drop(labels=['index','Table_flag'], axis=1, inplace=True) #'level_0',

# tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,fu2011d,3)

# #len(tmatches)

# # This finds 16 matches, which is exactly what we'd expect (there are 16 doubles in the table from Fu+2011)

# for i, j in zip(idx1, idx2):
#     #print("i:", i, "j:", j)
#     #print("Sep:", mcgurk2015t4.at[j, 'NIRC2sep(as)'])
#     #print("dV:", mcgurk2015t4.at[j, 'dV[OIII]'])
#     the_whills.at[i, 'Sep'] = fu2011d.at[j, 'Sep_as']
#     the_whills.at[i, 'Paper(s)'] += ' ; Fu+2011'
#     the_whills.at[i, 'BibCode(s)'] += ' ; 2011ApJ...733..103F' 
#     the_whills.at[i, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-637X/733/2/103'
#     the_whills.at[i, 'dV'] = fu2011d.at[j, 'dV']
#     the_whills.at[i, 'Notes'] += ' Fu+2011 companions within 3 arcseconds,'

# for index, row in the_whills.iterrows():
#     if index in idx1:
#         the_whills.at[index, 'Paper(s)'] += ' ; Fu+2012'
#         the_whills.at[index, 'BibCode(s)'] += ' ; 2012ApJ...745...67F' 
#         the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-637X/745/1/67'
#         the_whills.at[index, 'Notes'] += ' Companion(s) within 3 arcseconds.'



In [17]:
#guo2019

In [18]:
#Now for Gou+2019
guo2019 = pd.read_csv('Tables/Gou2019/Gou2019t2.csv', sep=',')

name_to_coords(guo2019,guo2019['Name'])
#guo2019objs = guo2019['Name'].tolist()
#print(len(guo2019objs))
coordconvert = SkyCoord(ra = guo2019['RA'], dec = guo2019['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
guo2019['RA1_deg'] = coordconvert.ra.degree
guo2019['Dec1_deg'] = coordconvert.dec.degree


# I think all we need to do here is append the Gou+2019 paper information for now. If need be, we can go back for \
# more later on...

tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,guo2019,5)

# verified that there are no matches to within 20''
print(len(tmatches))
#tmatches
for i, j in zip(idx1, idx2):
    the_whills.at[i, 'Paper(s)'] += ' ; Guo+2019'
    the_whills.at[i, 'BibCode(s)'] += ' ; 2019MNRAS.482.3288G' 
    the_whills.at[i, 'DOI(s)'] += ' ; https://doi.org/10.1093/mnras/sty2920'


#for i in the_whills['Paper(s)']:
#    if 'Guo+2019' in i:
#        print('True')
        
# Need to add notes about which one Guo+2019 considered to be the remaining best candidates....

# verified that this matches properly!


24


In [19]:
# Here we're loading in the Eracleous+2012 table(s)

erac2012 = ((Table.read('Tables/Eracleous2012/table1.dat', readme = 'Tables/Eracleous2012/ReadMe', format='ascii.cds')).to_pandas())#.drop(columns=['---'])

# Need to add in other info for the 68 candiates follow-up and the 14 specific candidates they discuss



erac2012.drop_duplicates(subset=['SDSS'], inplace=True)
erac2012.reset_index(drop=True, inplace=True)

erac2012['Name1'] = erac2012['SDSS']
erac2012['Name2'] = erac2012['SDSS']
erac2012['z1'] = erac2012['z']
erac2012['z2'] = erac2012['z']
erac2012['z1_type'] = "spec"
erac2012['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
name_to_coords(erac2012,erac2012['SDSS'])

# Converting the coordinates
coordconvert = SkyCoord(ra = erac2012['RA'], dec = erac2012['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
erac2012['RA1_deg'] = coordconvert.ra.degree
erac2012['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
erac2012['RA2'] = erac2012['RA']
erac2012['Dec2'] = erac2012['Dec']

erac2012['RA2_deg'] = erac2012['RA1_deg']
erac2012['Dec2_deg'] = erac2012['Dec1_deg']

# Adding details about the coordinates
erac2012['Equinox'] = "J2000"
erac2012['Coordinate_waveband'] = "Optical"
erac2012['Coordinate_Source'] = "SDSS"

erac2012['System Type'] = 'Binary AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
erac2012['Brightness1'] = erac2012['Vmag']
erac2012['Brightness_band1'] = 'V'
erac2012['Brightness_type1'] = 'apparent mag'

erac2012['Brightness2'] = erac2012['Vmag']
erac2012['Brightness_band2'] = 'V'
erac2012['Brightness_type2'] = 'apparent mag'

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
erac2012['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit


#erac2012['Sep(kpc)'] = erac2012['Sep']*((cosmo.arcsec_per_kpc_proper(erac2012['z']))**(-1))


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#erac2012['delta_z'] = erac2012['z1']-erac2012['z2']
erac2012['dV'] = (2.99e+5)*((1+erac2012['z1'])**2 - (1+erac2012['z2'])**2)/((1+erac2012['z1'])**2+(1+erac2012['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
erac2012['Selection Method'] = "Double-Peaked Broad Optical Spectroscopic Emission Lines" #DPSELs
erac2012['Confirmation Method'] = "-99"
erac2012['Paper(s)'] = "Eracleous+2012 ; Runnoe+2015 ; Runnoe+2017 ; Nguyen+2020"
erac2012['BibCode(s)'] = "2012ApJS..201...23E ; 2015ApJS..221....7R ; 2017MNRAS.468.1683R ; 2020ApJ...894..105N"
erac2012['DOI(s)'] = "doi:10.1088/0067-0049/201/2/23 ; https://doi.org/10.1088/0067-0049/221/1/7 ; https://doi.org/10.1093/mnras/stx452 ; https://doi.org/10.3847/1538-4357/ab88b5"

## And dropping any columns that we don't need....
#erac2012.drop(labels=['SDSS','f_SDSS','Vel','logL','Type','Q','zr','zh'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#erac2012 = erac2012[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#erac2012.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']
#


# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# Note Runnoe+ and Nyugen+ may have some info we want to include here, but for now I'm just listing their paper
# info since they looked at all of the same 88 objects

erac2012

,SDSS,z,Vmag,AV,VMag,Date,Instr,Exp,S/N,lam1,lam2,Name1,Name2,z1,z2,z1_type,z2_type,RA,Dec,RA1_deg,Dec1_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,dV,Selection Method,Confirmation Method,Paper(s),BibCode(s),DOI(s)
0,001224.02-102226.2,0.2287,17.07,0.127,-23.27,2001/08/20,S,3756,29,3093,7505,001224.02-102226.2,001224.02-102226.2,0.2287,0.2287,spec,spec,00:12:24.02,-10:22:26.2,3.100083,-10.373944,00:12:24.02,-10:22:26.2,3.100083,-10.373944,J2000,Optical,SDSS,Binary AGN Candidate,17.07,V,apparent mag,17.07,V,apparent mag,3,0.0,Double-Peaked Broad Optical Spectroscopic Emis...,-99,Eracleous+2012 ; Runnoe+2015 ; Runnoe+2017 ; N...,2012ApJS..201...23E ; 2015ApJS..221....7R ; 20...,doi:10.1088/0067-0049/201/2/23 ; https://doi.o...
1,002444.10+003221.3,0.4024,16.85,0.083,-24.86,2000/12/22,S,9900,88,2709,6594,002444.10+003221.3,002444.10+003221.3,0.4024,0.4024,spec,spec,00:24:44.10,+00:32:21.3,6.183750,0.539250,00:24:44.10,+00:32:21.3,6.183750,0.539250,J2000,Optical,SDSS,Binary AGN Candidate,16.85,V,apparent mag,16.85,V,apparent mag,3,0.0,Double-Peaked Broad Optical Spectroscopic Emis...,-99,Eracleous+2012 ; Runnoe+2015 ; Runnoe+2017 ; N...,2012ApJS..201...23E ; 2015ApJS..221....7R ; 20...,doi:10.1088/0067-0049/201/2/23 ; https://doi.o...
2,015530.01-085704.0,0.1648,16.84,0.080,-22.65,2001/09/16,S,2702,32,3263,7917,015530.01-085704.0,015530.01-085704.0,0.1648,0.1648,spec,spec,01:55:30.01,-08:57:04.0,28.875042,-8.951111,01:55:30.01,-08:57:04.0,28.875042,-8.951111,J2000,Optical,SDSS,Binary AGN Candidate,16.84,V,apparent mag,16.84,V,apparent mag,3,0.0,Double-Peaked Broad Optical Spectroscopic Emis...,-99,Eracleous+2012 ; Runnoe+2015 ; Runnoe+2017 ; N...,2012ApJS..201...23E ; 2015ApJS..221....7R ; 20...,doi:10.1088/0067-0049/201/2/23 ; https://doi.o...
3,020011.53-093126.2,0.3602,17.96,0.081,-23.46,2001/08/28,S,8104,34,2795,6780,020011.53-093126.2,020011.53-093126.2,0.3602,0.3602,spec,spec,02:00:11.53,-09:31:26.2,30.048042,-9.523944,02:00:11.53,-09:31:26.2,30.048042,-9.523944,J2000,Optical,SDSS,Binary AGN Candidate,17.96,V,apparent mag,17.96,V,apparent mag,3,0.0,Double-Peaked Broad Optical Spectroscopic Emis...,-99,Eracleous+2012 ; Runnoe+2015 ; Runnoe+2017 ; N...,2012ApJS..201...23E ; 2015ApJS..221....7R ; 20...,doi:10.1088/0067-0049/201/2/23 ; https://doi.o...
4,021259.59-003029.5,0.3945,17.87,0.117,-23.83,2000/09/29,S,2700,29,2714,6613,021259.59-003029.5,021259.59-003029.5,0.3945,0.3945,spec,spec,02:12:59.59,-00:30:29.5,33.248292,-0.508194,02:12:59.59,-00:30:29.5,33.248292,-0.508194,J2000,Optical,SDSS,Binary AGN Candidate,17.87,V,apparent mag,17.87,V,apparent mag,3,0.0,Double-Peaked Broad Optical Spectroscopic Emis...,-99,Eracleous+2012 ; Runnoe+2015 ; Runnoe+2017 ; N...,2012ApJS..201...23E ; 2015ApJS..221....7R ; 20...,doi:10.1088/0067-0049/201/2/23 ; https://doi.o...
5,022014.57-072859.1,0.2137,18.65,0.078,-21.47,2001/09/10,S,4501,11,3133,7598,022014.57-072859.1,022014.57-072859.1,0.2137,0.2137,spec,spec,02:20:14.57,-07:28:59.1,35.060708,-7.483083,02:20:14.57,-07:28:59.1,35.060708,-7.483083,J2000,Optical,SDSS,Binary AGN Candidate,18.65,V,apparent mag,18.65,V,apparent mag,3,0.0,Double-Peaked Broad Optical Spectroscopic Emis...,-99,Eracleous+2012 ; Runnoe+2015 ; Runnoe+2017 ; N...,2012ApJS..201...23E ; 2015ApJS..221....7R ; 20...,doi:10.1088/0067-0049/201/2/23 ; https://doi.o...
6,031715.87+425150.6,0.2356,19.03,0.616,-21.87,2003/12/03,S,13001,17,3075,7463,031715.87+425150.6,031715.87+425150.6,0.2356,0.2356,spec,spec,03:17:15.87,+42:51:50.6,49.316125,42.864056,03:17:15.87,+42:51:50.6,49.316125,42.864056,J2000,Optical,SDSS,Binary AGN Candidate,19.03,V,apparent mag,19.03,V,apparent mag,3,0.0,Double-Peaked Broad Optical Spectroscopic Emis...,-99,Eracleous+2012 ; Runnoe+2015 ; Runnoe+2017 ; N...,2012ApJS..201...23E ; 2015ApJS..221....7R ; 20...,doi:10.1088/0067-0049/201/2/23 ; https://doi.o...
7,074007

In [20]:
# There are 5 matches between the main table and the Eracleous+ table

tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,erac2012,5)

print(len(tmatches))

for i, j in zip(idx1, idx2):
    the_whills.at[i, 'Brightness1'] =  erac2012.at[j,'Brightness1']
    the_whills.at[i, 'Brightness_band1'] = erac2012.at[j,'Brightness_band1']
    the_whills.at[i, 'Brightness_type1'] = erac2012.at[j,'Brightness_type1']
    the_whills.at[i, 'Paper(s)'] += ' ; ' + str(erac2012.at[j,'Paper(s)']) 
    the_whills.at[i, 'BibCode(s)'] += ' ; ' + str(erac2012.at[j,'BibCode(s)']) 
    the_whills.at[i, 'DOI(s)'] += ' ; ' + str(erac2012.at[j,'DOI(s)']) 

#for i in the_whills['Paper(s)']:
#    if 'Eracleous+2012' in i:
#        print('True')

erac_nodups = erac2012.drop(idx2)

the_whills = pd.concat([the_whills,erac_nodups])



40


In [21]:
the_whills

,level_0,index,Name,RA,Dec,MJD,Plate,Fiber,z_NL,z_BL,Method,Class,Name1,Name2,z1,z2,z1_type,z2_type,RA1_deg,Dec1_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,dV,Selection Method,Confirmation Method,Paper(s),BibCode(s),DOI(s),Notes,Table_flag,Designation,z_sys,logM_1(M_sun),R_BLR(pc),E1Plate,E1Fiber,E1MJD,E2Plate,E2Fiber,E2MJD,Deltat(days),$V_{\rm ccf}^*$(km s^-1),$V_{\rm ccf}^*$(km s^-1).1,$V_{\rm ccf}^*$(km s^-1).2,$a_{\rm ccf}^*$(km s^-1 yr^-1),$a_{\rm ccf}^*$(km s^-1 yr^-1).1,$a_{\rm ccf}^*$(km s^-1 yr^-1).2,Category,delta_z,SDSS,z,VOFFP,e_VOFFP,VOFFC,Vmag,AV,VMag,Date,Instr,Exp,S/N,lam1,lam2
0,0.0,0,J0927+2943,09:27:12.65,+29:43:44.1,53389.0,1939.0,467.0,0.713,0.698,QG;QQ,B,J0927+2943,J0927+2943,0.7130,0.7130,spec,spec,141.802708,29.728917,09:27:12.65,+29:43:44.1,141.802708,29.728917,J2000,Optical,SDSS,Binary AGN Candidate,18.35,V,apparent mag,-100.00,-100,-100,3,0.0,Velocity Shifts between Narrow and Broad Lines,-99,Tsalmantza+2011 ; Decarli+2013 ; Lusso+2014 ; ...,2011ApJ...738...20T ; 2013MNRAS.433.1492D ; 20...,https://doi:10.1088/0004-637X/738/1/20 ; https...,Decarli+ find most objects do not undergo si...,Table1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,1,J0932+0318,09:32:01.60,+03:18:58.7,52254.0,568.0,39.0,0.420,0.401,QQ,B;D,J0932+0318,J0932+0318,0.4200,0.4200,spec,spec,143.006667,3.316306,09:32:01.60,+03:18:58.7,143.006667,3.316306,J2000,Optical,SDSS,Binary AGN Candidate,-100.00,-100,-100,-100.00,-100,-100,3,0.0,Velocity Shifts between Narrow and Broad Lines,-99,Tsalmantza+2011 ; Decarli+2013 ; Lusso+2014,2011ApJ...738...20T ; 2013MNRAS.433.1492D ; 20...,https://doi:10.1088/0004-637X/738/1/20 ; https...,Decarli+ find most objects do not undergo si...,Table1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,2,J1000+2233,10:00:21.80,+22:33:18.6,53737.0,2298.0,102.0,0.419,0.377,GQ,E;B,J1000+2233,J1000+2233,0.4190,0.4190,spec,spec,150.090833,22.555167,10:00:21.80,+22:33:18.6,150.090833,22.555167,J2000,Optical,SDSS,Binary AGN Candidate,-100.00,-100,-100,-100.00,-100,-100,3,0.0,Velocity Shifts between Narrow and Broad Lines,-99,Tsalmantza+2011 ; Decarli+2013 ; Lusso+2014,2011ApJ...738...20T ; 2013MNRAS.433.1492D ; 20...,https://doi:10.1088/0004-637X/738/1/20 ; https...,Decarli+ find most objects do not undergo si...,Table1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,3,J1012+2613,10:12:26.86,+26:13:27.3,53757.0,2347.0,513.0,0.378,0.351,QQ,E;B,J1012+2613,J1012+2613,0.3780,0.3780,spec,spec,153.111917,26.224250,10:12:26.86,+26:13:27.3,153.111917,26.224250,J2000,Optical,SDSS,Binary AGN Candidate,-100.00,-100,-100,-100.00,-100,-100,3,0.0,Velocity Shifts between Narrow and Broad Lines,-99,Tsalmantza+2011 ; Decarli+2013 ; Lusso+2014,2011ApJ...738...20T ; 2013MNRAS.433.1492D ; 20...,https://doi:10.1088/0004-637X/738/1/20 ; https...,Decarli+ find most objects do not undergo si...,Table1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,4,J1050+3456,10:50:41.36,+34:56:31.4,53431.0,2025.0,603.0,0.272,0.258,QQ;GQ,B,J1050+3456,J1050+3456,0.2720,0.2720,spec,spec,162.672333,34.942056,10:50:41.36,+34:56:31.4,162.672333,34.942056,J2000,Optical,SDSS,Binary AGN Candidate,18.95,V,apparent mag,-100.00,-100,-100,3,0.0,Velocity Shifts between Narrow and Broad Lines,-99,Tsalmantza+2011 ; Decarli+2013 ; Lusso+2014 ; ...,2011ApJ...738...20T ; 2013MNRAS.433.1492D ; 20...,https://doi:10.1088/0004-637X/738/1/20 ; https...,Decarli+ find most objects do not undergo si...,Table1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [22]:
# Here we're loading in the Ju+2013 table(s)

ju2013 = pd.read_csv('Tables/Ju2013/Ju2013.csv', sep=',')
#(M_sun),(yr),(km s^-1),(km s^-1),(km s^-1),(pc),
#Notes. (1) Object name in the SDSS catalog. (2) Redshift of the object from the SDSS catalog \
#(Bolton et al. 2012). (3) Absolute g-band magnitude. (4) BH mass of the object measured from the \
# Mg II widths (Shen et al. 2011b). (5) The time interval between multiple observations. (6) The velocity shift \
# of Mg II measured by cross-correlation over the whole wavelength range [1700 Aring, 4000 Aring]. \
# (7) The velocity shift of Mg II measured by cross-correlation over the wavelength range near Mg II \
# [2700 Aring, 2900 Aring]. (8) The uncertainties of velocity shifts reported from peak hunting of \
# cross-correlation functions. (9) The upper limit for the separation of the SMBH binary assuming a mass \
# ratio of q = 1. The true separations are equal to these values only when the observer sits at the ideal \
# position for watching velocity drifts: an edge-on orbit with the secondary BH just intersecting the line \
# connecting the primary BH and the observer. (10) The date when the last SDSS observation of the quasar was \
# taken, in the format of MM-DD-YYYY. * The more reliable candidates that pass the test cross-correlation over \
# [1700 Aring, 3800 Aring] (see Section 4.1). m: for these quasars, one of the two epoch spectra has a \
# "marginal" plate quality flagged in SDSS platelist. These plates get the "marginal" flag because either \
# the minimum S/N of all the fibers is smaller than 15.0 or the number of exposures is smaller than 3. b: \
# one of the two epoch spectra has a "bad" plate quality flag. This flag reflects the very poor seeing \
# conditions and bright sky when the data were taken, and may lead to large spectrophotometric uncertainty. \
# However, for this candidate, most of the spurious features are introduced in the red end of the spectrum, \
# far from the Mg II line. b+m: one of the two epoch spectra has a bad plate quality flag, while the other has \
# a marginal flag.

ju2013['Name1'] = ju2013['SDSS']
ju2013['Name2'] = ju2013['SDSS']
ju2013['z1'] = ju2013['z']
ju2013['z2'] = ju2013['z']
ju2013['z1_type'] = "spec"
ju2013['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(ju2013,ju2013['Designation'])

ju2013['Coordinates'] = ju2013['SDSS']#.str.slice(start=1) # Stripping the J
ju2013['RA_test'] = ju2013['Coordinates'].str.slice(start=1, stop=10) # Stripping the DEC parts 
ju2013['Dec_test'] = ju2013['Coordinates'].str.slice(start=10, stop=19) # Stripping the RA parts
ju2013['RA'] = ju2013['RA_test'].str.slice(start=0, stop=2)+":"+ju2013['RA_test'].str.slice(start=2, stop=4)+":"+ju2013['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
ju2013['Dec'] = ju2013['Dec_test'].str.slice(start=0, stop=3)+":"+ju2013['Dec_test'].str.slice(start=3, stop=5)+":"+ju2013['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
ju2013.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)

# Converting the coordinates
coordconvert = SkyCoord(ra = ju2013['RA'], dec = ju2013['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
ju2013['RA1_deg'] = coordconvert.ra.degree
ju2013['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
ju2013['RA2'] = ju2013['RA']
ju2013['Dec2'] = ju2013['Dec']

ju2013['RA2_deg'] = ju2013['RA1_deg']
ju2013['Dec2_deg'] = ju2013['Dec1_deg']

# Adding details about the coordinates
ju2013['Equinox'] = "J2000"
ju2013['Coordinate_waveband'] = "Optical"
ju2013['Coordinate_Source'] = "SDSS"

ju2013['System Type'] = 'Binary AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
ju2013['Brightness1'] = -100
ju2013['Brightness_band1'] = -100
ju2013['Brightness_type1'] = -100

ju2013['Brightness2'] = -100
ju2013['Brightness_band2'] = -100
ju2013['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
ju2013['Sep'] = 3 # arcseconds 
# Ju+'s method is sensitive to binaries with separations ~0.1pc 
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit


#ju2013['Sep(kpc)'] = ju2013['Sep']*((cosmo.arcsec_per_kpc_proper(ju2013['z']))**(-1))


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#ju2013['delta_z'] = ju2013['z1']-ju2013['z2']
ju2013['dV'] = (2.99e+5)*((1+ju2013['z1'])**2 - (1+ju2013['z2'])**2)/((1+ju2013['z1'])**2+(1+ju2013['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
ju2013['Selection Method'] = "Temporal Velocity Shifts in Mg II" #DPSELs
ju2013['Confirmation Method'] = "-99"
ju2013['Paper(s)'] = "Ju+2013"
ju2013['BibCode(s)'] = "2013ApJ...777...44J"
ju2013['DOI(s)'] = "https://doi.org/10.1088/0004-637X/777/1/44"
ju2013['Notes'] = " "


## And dropping any columns that we don't need....
#ju2013.drop(labels=['SDSS','f_SDSS','Vel','logL','Type','Q','zr','zh'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#ju2013 = ju2013[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#ju2013.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']
#


#ju2013

In [23]:
tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,ju2013,5)

print(len(tmatches))

for i, j in zip(idx1, idx2):
    the_whills.at[i, 'Selection Method'] += ' ; Temporal Velocity Shifts in Mg II' 
    the_whills.at[i, 'Paper(s)'] += ' ; Ju+2013' 
    the_whills.at[i, 'BibCode(s)'] += ' ; 2013ApJ...777...44J'
    the_whills.at[i, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-637X/777/1/44'

for i in the_whills['Paper(s)']:
    if 'Ju+2013' in i:
        print('True')

ju_nodups = ju2013.drop(idx2)

the_whills = pd.concat([the_whills,ju_nodups])

# verified that this matching process works!


2
True


In [24]:
#the_whills

In [25]:
# Loading in here the results from Wang+2017, a follow-up work for Ju+2013

wang2017t1 = pd.read_csv('Tables/Wang2017/Wang2017.csv',sep=',')

#objs = wang2017t1['SDSSID'].tolist()
name_to_coords(wang2017t1,wang2017t1['SDSSID'])
# Converting the coordinates
coordconvert = SkyCoord(ra = wang2017t1['RA'], dec = wang2017t1['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
wang2017t1['RA1_deg'] = coordconvert.ra.degree
wang2017t1['Dec1_deg'] = coordconvert.dec.degree


tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,wang2017t1,10)
print(len(tmatches))

for i, j in zip(idx1, idx2):
    #the_whills.at[i, 'Selection Method'] += ' ; Temporal Velocity Shifts in Mg II' 
    the_whills.at[i, 'Paper(s)'] += ' ; Wang+2017' 
    the_whills.at[i, 'BibCode(s)'] += ' ; 2017ApJ...834..129W'
    the_whills.at[i, 'DOI(s)'] += ' ; https://doi.org/ 10.3847/1538-4357/834/2/129'

for i in the_whills['Paper(s)']:
    if 'Wang+2017' in i:
        print('True')

#ju_nodups = wang2017t1.drop(idx2)
#
#the_whills = pd.concat([the_whills,ju_nodups])


# verified, I think this is working properly.
# Note: Wang+ followed up 21 targets from Ju+2013's overarching analysis. NOT only the 9 candidates that Ju flagged.
# In fact, some of these candiates were not examined again by Wang+ because they were not observed again by SDSS

6
True
True
True


In [26]:
# ******************************
# --> These are all from the Mg II cross correlation only. We'll need to load in that table from Ju+2013!
# ******************************
    
wang2017 = pd.read_csv('Tables/Wang2017/Wang2017_t3.csv', sep=',')

wang2017 = wang2017[wang2017['SDSSID']=="J163709.31+414030.8"]
# here  we are limiting to the conservative choice Wang+ made where velocities ~>450 are considered due to binary motion

wang2017['Name'] = wang2017['SDSSID']
wang2017['Name2'] = wang2017['SDSSID']
wang2017['z1'] = wang2017['z']
wang2017['z2'] = wang2017['z']
wang2017['z1_type'] = "spec"
wang2017['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(wang2017,wang2017['Designation'])

wang2017['Coordinates'] = wang2017['SDSSID']#.str.slice(start=1) # Stripping the J
wang2017['RA_test'] = wang2017['Coordinates'].str.slice(start=1, stop=10) # Stripping the DEC parts 
wang2017['Dec_test'] = wang2017['Coordinates'].str.slice(start=10, stop=19) # Stripping the RA parts
wang2017['RA'] = wang2017['RA_test'].str.slice(start=0, stop=2)+":"+wang2017['RA_test'].str.slice(start=2, stop=4)+":"+wang2017['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
wang2017['Dec'] = wang2017['Dec_test'].str.slice(start=0, stop=3)+":"+wang2017['Dec_test'].str.slice(start=3, stop=5)+":"+wang2017['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
wang2017.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)

# Converting the coordinates
coordconvert = SkyCoord(ra = wang2017['RA'], dec = wang2017['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
wang2017['RA1_deg'] = coordconvert.ra.degree
wang2017['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
wang2017['RA2'] = wang2017['RA']
wang2017['Dec2'] = wang2017['Dec']

wang2017['RA2_deg'] = wang2017['RA1_deg']
wang2017['Dec2_deg'] = wang2017['Dec1_deg']

# Adding details about the coordinates
wang2017['Equinox'] = "J2000"
wang2017['Coordinate_waveband'] = "Optical"
wang2017['Coordinate_Source'] = "SDSS"

wang2017['System Type'] = 'Binary AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
wang2017['Brightness1'] = -100
wang2017['Brightness_band1'] = -100
wang2017['Brightness_type1'] = -100

wang2017['Brightness2'] = -100
wang2017['Brightness_band2'] = -100
wang2017['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
#wang2017['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit


#wang2017['Sep(kpc)'] = wang2017['Sep']*((cosmo.arcsec_per_kpc_proper(wang2017['z']))**(-1))


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#wang2017['delta_z'] = wang2017['z1']-wang2017['z2']
wang2017['dV'] = (2.99e+5)*((1+wang2017['z1'])**2 - (1+wang2017['z2'])**2)/((1+wang2017['z1'])**2+(1+wang2017['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
wang2017['Selection Method'] = "Velocity Offset Emission Lines" #DPSELs
wang2017['Confirmation Method'] = "-99"
wang2017['Paper(s)'] = "Wang+2017"
wang2017['BibCode(s)'] = "2017ApJ...834..129W"
wang2017['DOI(s)'] = "https://doi.org/ 10.3847/1538-4357/834/2/129"

## And dropping any columns that we don't need....
#wang2017.drop(labels=['SDSS','f_SDSS','Vel','logL','Type','Q','zr','zh'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#wang2017 = wang2017[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#wang2017.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']
#

# Eventually we need to go back and flag the single object that is favored by them and flag the others as disfavored.

wang2017

,SDSSID,z,log10MBH,delta_t,delta_V,aLoS,Name,Name2,z1,z2,z1_type,z2_type,RA,Dec,RA1_deg,Dec1_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,dV,Selection Method,Confirmation Method,Paper(s),BibCode(s),DOI(s)
13,J163709.31+414030.8,0.76,9.76,10.99,448,40,J163709.31+414030.8,J163709.31+414030.8,0.76,0.76,spec,spec,16:37:09.31,+41:40:30.8,249.288792,41.675222,16:37:09.31,+41:40:30.8,249.288792,41.675222,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,Velocity Offset Emission Lines,-99,Wang+2017,2017ApJ...834..129W,https://doi.org/ 10.3847/1538-4357/834/2/129


In [27]:
tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,wang2017,5)
# No matches with the single object in Wang+'s table 3 and the rest of our overarching table

#print(len(tmatches))

the_whills = pd.concat([the_whills,wang2017])

# verified; works

In [28]:
# Adding in the recoil candidates from Kim here...

#Col 1: object name. Col 2: redshift. Col 3: SDSS r magnitude. Col 4: IRAS ADDSCAN/SCANPI 60 μm flux. Col 5: IRAS ADDSCAN/SCANPI 100 μm
#flux. Col 6: far-infrared luminosity. Col 7: line-of-sight velocity offset from Hα broad line. Col 8: estimated spatial offsets in arcseconds and kiloparsecs assuming an
#AGN lifetime of 10 Myr. Col 9: line-of-sight velocity offset from Hβ broad line. Col 10: velocity dispersion. Col 11: black hole masses estimated from virial method.
#Col 12: Eddington ratios calculated from virial black hole masses.

kim2016 = pd.read_csv('Tables/Kim2016/Kim2016.csv', sep=',')

kim2016['Name1'] = kim2016['Name']
kim2016['Name2'] = kim2016['Name']
kim2016['z1'] = kim2016['z']
kim2016['z2'] = kim2016['z']
kim2016['z1_type'] = "spec"
kim2016['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
name_to_coords(kim2016,kim2016['Name'])

# Converting the coordinates
coordconvert = SkyCoord(ra = kim2016['RA'], dec = kim2016['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
kim2016['RA1_deg'] = coordconvert.ra.degree
kim2016['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
kim2016['RA2'] = kim2016['RA']
kim2016['Dec2'] = kim2016['Dec']

kim2016['RA2_deg'] = kim2016['RA1_deg']
kim2016['Dec2_deg'] = kim2016['Dec1_deg']

# Adding details about the coordinates
kim2016['Equinox'] = "J2000"
kim2016['Coordinate_waveband'] = "Optical"
kim2016['Coordinate_Source'] = "SDSS"

kim2016['System Type'] = 'Recoil Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
kim2016['Brightness1'] = -100
kim2016['Brightness_band1'] = -100
kim2016['Brightness_type1'] = -100

kim2016['Brightness2'] = -100
kim2016['Brightness_band2'] = -100
kim2016['Brightness_type2'] = -100


# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
kim2016['Sep'] = kim2016['Spatial Offset (as)'] # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit

#kim2016['Sep(kpc)'] = kim2016['Sep']*((cosmo.arcsec_per_kpc_proper(kim2016['z']))**(-1))

# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#kim2016['delta_z'] = kim2016['z1']-kim2016['z2']
kim2016['dV'] = kim2016['delta_V(Halpha)'] #(2.99e+5)*((1+kim2016['z1'])**2 - (1+kim2016['z2'])**2)/((1+kim2016['z1'])**2+(1+kim2016['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
kim2016['Selection Method'] = "Offset Broad Lines" #DPSELs
kim2016['Confirmation Method'] = "-99"
kim2016['Paper(s)'] = "Kim+2016"
kim2016['BibCode(s)'] = "2016ApJ...824..122K"
kim2016['DOI(s)'] = "https://doi.org/10.3847/0004-637X/824/2/122"

## And dropping any columns that we don't need....
#kim2016.drop(labels=['SDSS','f_SDSS','Vel','logL','Type','Q','zr','zh'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#kim2016 = kim2016[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#kim2016.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']
#

kim2016


,Name,z,r,f60,f100,logLFIR,delta_V(Halpha),delta_V(Halpha)_err,Spatial Offset (as),Spatial Offset (kpc),delta_V(Hbeta),delta_V(Hbeta)_err,Name1,Name2,z1,z2,z1_type,z2_type,RA,Dec,RA1_deg,Dec1_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,dV,Selection Method,Confirmation Method,Paper(s),BibCode(s),DOI(s)
0,075819.69+421935.1,0.212,17.01,0.33,0.54,11.98,-154,10,0.46,1.57,-226,25,075819.69+421935.1,075819.69+421935.1,0.212,0.212,spec,spec,07:58:19.69,+42:19:35.1,119.582042,42.326417,07:58:19.69,+42:19:35.1,119.582042,42.326417,J2000,Optical,SDSS,Recoil Candidate,-100,-100,-100,-100,-100,-100,0.46,-154,Offset Broad Lines,-99,Kim+2016,2016ApJ...824..122K,https://doi.org/10.3847/0004-637X/824/2/122
1,080101.41+184840.7,0.140,16.62,0.08,0.62,11.36,-87,10,0.36,0.89,-110,4,080101.41+184840.7,080101.41+184840.7,0.140,0.140,spec,spec,08:01:01.41,+18:48:40.7,120.255875,18.811306,08:01:01.41,+18:48:40.7,120.255875,18.811306,J2000,Optical,SDSS,Recoil Candidate,-100,-100,-100,-100,-100,-100,0.36,-87,Offset Broad Lines,-99,Kim+2016,2016ApJ...824..122K,https://doi.org/10.3847/0004-637X/824/2/122
2,080742.46+375332.1,0.230,18.18,0.06,0.15,11.40,-76,5,0.21,0.78,-105,10,080742.46+375332.1,080742.46+375332.1,0.230,0.230,spec,spec,08:07:42.46,+37:53:32.1,121.926917,37.892250,08:07:42.46,+37:53:32.1,121.926917,37.892250,J2000,Optical,SDSS,Recoil Candidate,-100,-100,-100,-100,-100,-100,0.21,-76,Offset Broad Lines,-99,Kim+2016,2016ApJ...824..122K,https://doi.org/10.3847/0004-637X/824/2/122
3,085946.35+274534.8,0.245,17.71,0.15,0.26,11.79,-85,10,0.23,0.87,-105,5,085946.35+274534.8,085946.35+274534.8,0.245,0.245,spec,spec,08:59:46.35,+27:45:34.8,134.943125,27.759667,08:59:46.35,+27:45:34.8,134.943125,27.759667,J2000,Optical,SDSS,Recoil Candidate,-100,-100,-100,-100,-100,-100,0.23,-85,Offset Broad Lines,-99,Kim+2016,2016ApJ...824..122K,https://doi.org/10.3847/0004-637X/824/2/122
4,090654.47+391455.3,0.241,17.78,0.12,0.12,11.60,-137,36,0.37,1.40,-101,23,090654.47+391455.3,090654.47+391455.3,0.241,0.241,spec,spec,09:06:54.47,+39:14:55.3,136.726958,39.248694,09:06:54.47,+39:14:55.3,136.726958,39.248694,J2000,Optical,SDSS,Recoil Candidate,-100,-100,-100,-100,-100,-100,0.37,-137,Offset Broad Lines,-99,Kim+2016,2016ApJ...824..122K,https://doi.org/10.3847/0004-637X/824/2/122
5,091617.53+303037.9,0.215,18.40,0.10,0.17,11.48,-208,17,0.61,2.12,-236,22,091617.53+303037.9,091617.53+303037.9,0.215,0.215,spec,spec,09:16:17.53,+30:30:37.9,139.073042,30.510528,09:16:17.53,+30:30:37.9,139.073042,30.510528,J2000,Optical,SDSS,Recoil Candidate,-100,-100,-100,-100,-100,-100,0.61,-208,Offset Broad Lines,-99,Kim+2016,2016ApJ...824..122K,https://doi.org/10.3847/0004-637X/824/2/122
6,092247.03+512038.0,0.161,17.39,0.16,0.31,11.43,-212,13,0.78,2.17,-215,13,092247.03+512038.0,092247.03+512038.0,0.161,0.161,spec,spec,09:22:47.03,+51:20:38.0,140.695958,51.343889,09:22:47.03,+51:20:38.0,140.695958,51.343889,J2000,Optical,SDSS,Recoil Candidate,-100,-100,-100,-100,-100,-100,0.78,-212,Offset Broad Lines,-99,Kim+2016,2016ApJ...824..122K,https://doi.org/10.3847/0004-637X/824/2/122
7,105752.69+105037.9,0.221,17.78,K,K,K,-91,0,0.26,0.93,-137,13,105752.69+105037.9,105752.69+105037.9,0.221,0.221,spec,spec,10:57:52.69,+10:50:37.9,164.469542,10.843861,10:57:52.69,+10:50:37.9,164.469542,10.843861,J2000,Optical,SDSS,Recoil Candidate,-100,-100,-100,-100,-100,-100,0.26,-91,Offset Broad Lines,-99,Kim+2016,2016ApJ...824..122K,https://doi.org/10.3847/0004-637X/824/2/122
8,115341.16+101754.0,0.161,16.37,0.25,0.38,11.58,-202,5,0.74,2.07,-189,25,115341.16+101754.0,115341.16+101754.0,0.161,0.161,spec,spec,11:53:41.16,+10:17:54.0,178.421500,10.298333,11:53:41.16,+10:17:54.0,178.421500,10.298333,J2000,Optical,SDSS,Recoil Candidate,-100,-100,-100,-100,-100,-100,0.74,-202,Offset Broad Lines,-99,Kim+2016,2016ApJ...824..122K,https://doi.org/10.3847/0004-637X/824/2/122
9,11563

In [29]:
# 2 matches found

tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,kim2016,5)
print(len(tmatches))

for i, j in zip(idx1, idx2):
    the_whills.at[i, 'Selection Method'] += ' ; Offset Broad Lines'
    the_whills.at[i, 'System Type'] += ' ; ' + str(kim2016.at[j, 'System Type'])
    the_whills.at[i, 'Sep'] += kim2016.at[j, 'Spatial Offset (as)']
    the_whills.at[i, 'dV'] += kim2016.at[j, 'delta_V(Halpha)']
    the_whills.at[i, 'Paper(s)'] += ' ; Kim+2016' 
    the_whills.at[i, 'BibCode(s)'] += ' ; 2016ApJ...824..122K'
    the_whills.at[i, 'DOI(s)'] += ' ; https://doi.org/10.3847/0004-637X/824/2/122'

#for i in the_whills['Paper(s)']:
#    if 'Kim=2016' in i:
#        print('True')

kim_nodups = kim2016.drop(idx2)

the_whills = pd.concat([the_whills,kim_nodups])

# verified that this matching process is working

4


In [35]:
the_whills

,RA,Dec,Name1,Name2,z1,z2,z1_type,z2_type,RA1_deg,Dec1_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,dV,Selection Method,Confirmation Method,Paper(s),BibCode(s),DOI(s),Notes
0,09:27:12.65,+29:43:44.1,J0927+2943,J0927+2943,0.7130,0.7130,spec,spec,141.802708,29.728917,09:27:12.65,+29:43:44.1,141.802708,29.728917,J2000,Optical,SDSS,Binary AGN Candidate,18.35,V,apparent mag,-100.00,-100,-100,3.00,0.0,Velocity Shifts between Narrow and Broad Lines,-99,Tsalmantza+2011 ; Decarli+2013 ; Lusso+2014 ; ...,2011ApJ...738...20T ; 2013MNRAS.433.1492D ; 20...,https://doi:10.1088/0004-637X/738/1/20 ; https...,Decarli+ find most objects do not undergo si...
1,09:32:01.60,+03:18:58.7,J0932+0318,J0932+0318,0.4200,0.4200,spec,spec,143.006667,3.316306,09:32:01.60,+03:18:58.7,143.006667,3.316306,J2000,Optical,SDSS,Binary AGN Candidate,-100.00,-100,-100,-100.00,-100,-100,3.00,0.0,Velocity Shifts between Narrow and Broad Lines,-99,Tsalmantza+2011 ; Decarli+2013 ; Lusso+2014,2011ApJ...738...20T ; 2013MNRAS.433.1492D ; 20...,https://doi:10.1088/0004-637X/738/1/20 ; https...,Decarli+ find most objects do not undergo si...
2,10:00:21.80,+22:33:18.6,J1000+2233,J1000+2233,0.4190,0.4190,spec,spec,150.090833,22.555167,10:00:21.80,+22:33:18.6,150.090833,22.555167,J2000,Optical,SDSS,Binary AGN Candidate,-100.00,-100,-100,-100.00,-100,-100,3.00,0.0,Velocity Shifts between Narrow and Broad Lines,-99,Tsalmantza+2011 ; Decarli+2013 ; Lusso+2014,2011ApJ...738...20T ; 2013MNRAS.433.1492D ; 20...,https://doi:10.1088/0004-637X/738/1/20 ; https...,Decarli+ find most objects do not undergo si...
3,10:12:26.86,+26:13:27.3,J1012+2613,J1012+2613,0.3780,0.3780,spec,spec,153.111917,26.224250,10:12:26.86,+26:13:27.3,153.111917,26.224250,J2000,Optical,SDSS,Binary AGN Candidate,-100.00,-100,-100,-100.00,-100,-100,3.00,0.0,Velocity Shifts between Narrow and Broad Lines,-99,Tsalmantza+2011 ; Decarli+2013 ; Lusso+2014,2011ApJ...738...20T ; 2013MNRAS.433.1492D ; 20...,https://doi:10.1088/0004-637X/738/1/20 ; https...,Decarli+ find most objects do not undergo si...
4,10:50:41.36,+34:56:31.4,J1050+3456,J1050+3456,0.2720,0.2720,spec,spec,162.672333,34.942056,10:50:41.36,+34:56:31.4,162.672333,34.942056,J2000,Optical,SDSS,Binary AGN Candidate,18.95,V,apparent mag,-100.00,-100,-100,3.00,0.0,Velocity Shifts between Narrow and Broad Lines,-99,Tsalmantza+2011 ; Decarli+2013 ; Lusso+2014 ; ...,2011ApJ...738...20T ; 2013MNRAS.433.1492D ; 20...,https://doi:10.1088/0004-637X/738/1/20 ; https...,Decarli+ find most objects do not undergo si...
5,11:54:49.42,+01:34:43.6,J1154+0134,J1154+0134,0.4690,0.4690,spec,spec,178.705917,1.578778,11:54:49.42,+01:34:43.6,178.705917,1.578778,J2000,Optical,SDSS,Binary AGN Candidate,17.94,V,apparent mag,-100.00,-100,-100,3.00,0.0,Velocity Shifts between Narrow and Broad Lines,-99,Tsalmantza+2011 ; Decarli+2013 ; Lusso+2014 ; ...,2011ApJ...738...20T ; 2013MNRAS.433.1492D ; 20...,https://doi:10.1088/0004-637X/738/1/20 ; https...,Decarli+ find most objects do not undergo si...
6,15:36:36.22,+04:41:27.0,J1536+0441,J1536+0441,0.3890,0.3890,spec,spec,234.150917,4.690833,15:36:36.22,+04:41:27.0,234.150917,4.690833,J2000,Optical,SDSS,Binary AGN Candidate,17.07,V,apparent mag,-100.00,-100,-100,3.00,0.0,Velocity Shifts between Narrow and Broad Lines,-99,Tsalmantza+2011 ; Decarli+2013 ; Lusso+2014 ; ...,2011ApJ...738...20T ; 2013MNRAS.433.1492D ; 20...,https://doi:10.1088/0004-637X/738/1/20 ; https...,Decarli+ find most objects do not undergo si...
7,15:39:08.09,+33:33:28.0,J1539+3333,J1539+3333,0.2260,0.2260,spec,spec,234.783708,33.557778,15:39:08.09,+33:33:28.0,234.783708,33.557778,J2000,Optical,SDSS,Binary AGN Candidate,-100.00,-100,-100,-100.00,-100,-100,3.00,0.0,Velocity Shifts between Narrow and Broad Lines,-99,Tsalmantza+2011 ; Decarli+2013 ; Lusso+2014,2011ApJ...738...20T ; 2013MNRAS.433.1492D ; 20...,https://doi:10.1088/0004-637

In [32]:
# now formatting the final table...
the_whills.drop(labels=['level_0','index','Category','delta_z','SDSS','z','VOFFP','e_VOFFP','VOFFC','Vmag',\
                        'AV','VMag','Date','Instr','Exp','S/N','lam1','lam2','Ju2013_flag','M_abs','log_10M_BH',\
                        'Deltat','DeltaV_1','DeltaV_2','Error','r_max','Last_Observation','SDSSID','log10MBH',\
                        'delta_t','delta_V','aLoS','r','f60','f100','logLFIR','delta_V(Halpha)',\
                        'delta_V(Halpha)_err','Spatial Offset (as)','Spatial Offset (kpc)','delta_V(Hbeta)',\
                        'delta_V(Hbeta)_err','Table_flag','Designation','z_sys','logM_1(M_sun)','R_BLR(pc)',\
                        'E1Plate','E1Fiber','E1MJD','E2Plate','E2Fiber','E2MJD','Deltat(days)',\
                        'Category'], axis=1, inplace=True)



In [34]:
the_whills.drop(labels=['Name','MJD','Plate','Fiber','z_NL','z_BL','Method','Class'], axis=1, inplace=True)

the_whills = the_whills.iloc[:, :-6]



In [36]:
# now saving the table...
the_whills.to_csv('Binary_broadline_tables_DR1.csv', sep=',', index=False)



In [125]:
#for i in the_whills['Paper(s)']:
#    if 'Kim+2016' in i:
#        print('True')

In [127]:
#the_whills

In [ ]:
###

# HENNAWI 2013 IS PROJECTED QUASAR PAIRS WITH VELOCITY DIFFERENCES >2000 KM S^-1, NO NEED TO INCLUDE.
# Prochaska+2013 are also projected pairs with >2000 km s^-1
# Lau+2018 and Lau+2016 required velocity differences >3000 km s^-1
# Rubni 2015 expunged all systems within 5000 km s^-1 of the foreground quasar

In [ ]:
# This needs to get moved to a miscellaneous notebook
# Loading in Mezcua+2014 here

#mezcua2014 = pd.read_csv('Tables/Mezcua2014/Mezcua2014.csv', sep=',')

# We need to gather coordinates, redshifts, and other information....


#mezcua2014['Name1'] = mezcua2014['SDSS Designation']
#mezcua2014['Name2'] = mezcua2014['SDSS Designation']
#mezcua2014['z'] = -99

#mezcua2014['z1'] = mezcua2014['z']
#mezcua2014['z2'] = mezcua2014['z']
#mezcua2014['z1_type'] = "spec"
#mezcua2014['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(mezcua2014,mezcua2014['Designation'])

#mezcua2014['Coordinates'] = mezcua2014['SDSS Designation']#.str.slice(start=1) # Stripping the J
#mezcua2014['RA_test'] = mezcua2014['Coordinates'].str.slice(start=0, stop=9) # Stripping the DEC parts 
#mezcua2014['Dec_test'] = mezcua2014['Coordinates'].str.slice(start=9, stop=19) # Stripping the RA parts
#mezcua2014['RA'] = mezcua2014['RA_test'].str.slice(start=0, stop=2)+":"+mezcua2014['RA_test'].str.slice(start=2, stop=4)+":"+mezcua2014['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
#mezcua2014['Dec'] = mezcua2014['Dec_test'].str.slice(start=0, stop=3)+":"+mezcua2014['Dec_test'].str.slice(start=3, stop=5)+":"+mezcua2014['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
#mezcua2014.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
#
## Converting the coordinates
#coordconvert = SkyCoord(ra = mezcua2014['RA'], dec = mezcua2014['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
#mezcua2014['RA_deg'] = coordconvert.ra.degree
#mezcua2014['Dec_deg'] = coordconvert.dec.degree
#
## Adding in a second set of coordinates for the 'secondary'
#mezcua2014['RA2'] = mezcua2014['RA']
#mezcua2014['Dec2'] = mezcua2014['Dec']
#
#mezcua2014['RA2_deg'] = mezcua2014['RA_deg']
#mezcua2014['Dec2_deg'] = mezcua2014['Dec_deg']

# Adding details about the coordinates
#mezcua2014['Equinox'] = "J2000"
#mezcua2014['Coordinate_waveband'] = "Optical"
#mezcua2014['Coordinate_Source'] = "SDSS"
#
#mezcua2014['System Type'] = 'Dual AGN Candidate'
#
## Adding in some columns that we'll population via a Simbad or Ned search later
#mezcua2014['Brightness1'] = -100
#mezcua2014['Brightness_band1'] = -100
#mezcua2014['Brightness_type1'] = -100
#
#mezcua2014['Brightness2'] = -100
#mezcua2014['Brightness_band2'] = -100
#mezcua2014['Brightness_type2'] = -100
#
## Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
## of order ~1 kpc or less, but is not currently determined.
#mezcua2014['Sep'] = 3 # arcseconds
## Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
## fiber as an upper limit
#
##mezcua2014['Sep(kpc)'] = mezcua2014['Sep']*((cosmo.arcsec_per_kpc_proper(mezcua2014['z']))**(-1))
#
## For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
##mezcua2014['delta_z'] = mezcua2014['z1']-mezcua2014['z2']
##mezcua2014['dV'] = (2.99e+5)*((1+mezcua2014['z1'])**2 - (1+mezcua2014['z2'])**2)/((1+mezcua2014['z1'])**2+(1+mezcua2014['z2'])**2)
## dV will be zero until we include follow-up observations that show separate redshifts

## Adding information about the paper and the selection method
#mezcua2014['Selection Method'] = "Projected Separation" #DPSELs
#mezcua2014['Confirmation Method'] = "-99"
#mezcua2014['Paper(s)'] = "Mezcua+2014"
#mezcua2014['BibCode(s)'] = "2014ApJ...784...16M"
#mezcua2014['DOI(s)'] = "https://doi.org/10.1088/0004-637X/784/1/16"
#
### And dropping any columns that we don't need....
##mezcua2014.drop(labels=['SDSS','f_SDSS','Vel','logL','Type','Q','zr','zh'],\
##              axis=1, inplace=True)
#
#mezcua2014